# Mineração de dados

Nesse trabalho estamos retirando informações dos dados da base de dados **Census Income**.

Como metodologia, estamos aplicando o processo de **KDD (Knowledge Discovery in Databases)**, onde teremos as seguintes etapas:

- **Seleção dos dados**:  
  Escolha dos atributos relevantes da base Census Income para o problema proposto.

- **Pré-processamento**:  
  Tratamento dos dados, incluindo:
  - Conversão de variáveis categóricas em formato numérico.
  - Normalização de variáveis numéricas.

- **Transformação dos dados (Feature Engineering)**:  
  Criação de novas variáveis para enriquecer o conjunto de dados, como:
  - Criação de faixas etárias a partir da variável `age`.
  - Tempo Livre(`free_time`).
  - Proporção de anos estudados(`education_age_ratio`).
  - Cálculo de `capital_net` (`capital-gain` - `capital-loss`).
  - Simplificação do estado civil (`is_married`).

- **Mineração de dados**:  
  Aplicação de algoritmos de Machine Learning para identificar padrões e realizar a tarefa de classificação da renda (`<=50K` ou `>50K`).

- **Interpretação/Evaluação dos resultados**:  
  Análise dos resultados obtidos pelos modelos, avaliando métricas como: 
  - acurácia.
  - precisão.
  - recall.
  - F1-score.

---

**Autores**: Pedro Bianchini de Quadros e Lucas Fernando Assunção Cavalherie  
**Fonte**: [Census Income Dataset - UCI](https://archive.ics.uci.edu/dataset/20/census+income)

## Importando Bibliotecas Necessarias para analise dos dados

In [1]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

## Importando Banco de Dados

In [2]:
adult = fetch_ucirepo(id=2)

In [3]:
# data (as pandas dataframes)
df = adult.data.features.copy()
df['income'] = adult.data.targets


print(df.shape)

(48842, 15)


In [4]:
# metadata
adult.metadata

{'uci_id': 2,
 'name': 'Adult',
 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult',
 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv',
 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ',
 'area': 'Social Science',
 'tasks': ['Classification'],
 'characteristics': ['Multivariate'],
 'num_instances': 48842,
 'num_features': 14,
 'feature_types': ['Categorical', 'Integer'],
 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'],
 'target_col': ['income'],
 'index_col': None,
 'has_missing_values': 'yes',
 'missing_values_symbol': 'NaN',
 'year_of_dataset_creation': 1996,
 'last_updated': 'Tue Sep 24 2024',
 'dataset_doi': '10.24432/C5XW20',
 'creators': ['Barry Becker', 'Ronny Kohavi'],
 'intro_paper': None,
 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was ex

In [5]:
# variable information
adult.variables

,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,N/A,None,no
1,workclass,Feature,Categorical,Income,"Private, Self-emp-not-inc, Self-emp-inc, Feder...",None,yes
2,fnlwgt,Feature,Integer,None,None,None,no
3,education,Feature,Categorical,Education Level,"Bachelors, Some-college, 11th, HS-grad, Prof-...",None,no
4,education-num,Feature,Integer,Education Level,None,None,no
5,marital-status,Feature,Categorical,Other,"Married-civ-spouse, Divorced, Never-married, S...",None,no
6,occupation,Feature,Categorical,Other,"Tech-support, Craft-repair, Other-service, Sal...",None,yes
7,relationship,Feature,Categorical,Other,"Wife, Own-child, Husband, Not-in-family, Other...",None,no
8,race,Feature,Categorical,Race,"White, Asian-Pac-Islander, Amer-Indian-Eskimo,...",None,no
9,sex,Feature,Binary,Sex,"Female, Male.",None,no


## Pre-Processamento dos dados

In [6]:
def verificar_nulos(df):
    """
    Verifica se existem valores nulos em um DataFrame ou Series do pandas.

    Args:
        df (pd.DataFrame ou pd.Series): Objeto pandas a ser verificado.

    Returns:
        True se houver nulos, False caso contrário.
    """
    if not isinstance(df, (pd.DataFrame, pd.Series)):
        raise TypeError("O objeto precisa ser um pandas DataFrame ou Series.")
    return df.isnull().values.any()

def percentual_nulos(df):
    """
    Mostra o percentual de valores nulos por coluna de um DataFrame.

    Args:
        df (pd.DataFrame): O DataFrame a ser analisado.

    Returns:
        pd.Series: Série com o percentual de nulos por coluna.
    """
    return (df.isnull().mean() * 100).sort_values(ascending=False)

In [7]:
verificar_nulos(df)

np.True_

In [8]:
percentual_nulos(df)

occupation        1.977806
workclass         1.971664
native-country    0.560993
fnlwgt            0.000000
education         0.000000
education-num     0.000000
age               0.000000
marital-status    0.000000
relationship      0.000000
sex               0.000000
race              0.000000
capital-gain      0.000000
capital-loss      0.000000
hours-per-week    0.000000
income            0.000000
dtype: float64

In [9]:
# Remove linhas com valores nulos
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

## Transformação dos dados

### Feature Engineering

Criação da faixa etária

| Faixa | Idade | 
|---------|-------|
| Jovem |(18–25)|
| Adulto |(26–45)|
| Meia-idade |(46–65)|
| Idoso|(66+)|

In [10]:
df['age_group'] = pd.cut(df['age'], bins=[0,25,45,65,100], labels=['Young','Adult','Middle-Aged','Senior'])

"Capital líquido"

In [11]:
df['capital_net'] = df['capital-gain'] - df['capital-loss']

Proporção de anos estudados por idade

In [12]:
df['education_age_ratio'] = df['education-num'] / df['age']

Tempo livre

In [13]:
df['free_time'] = 168 - df['hours-per-week']

Casado ou não

In [14]:
df['is_married'] = df['marital-status'].apply(lambda x: 1 if x == 'Married-civ-spouse' else 0)

Obs: Posso combinar mais features, ver dedpois com o Lucas

### Encoding de Features

In [ ]:
# TODO: Vou adicionar aqui, a parte de LabelEncoder/OneHotEncoder

### Algoritmos de Scaling

In [ ]:
# TODO: Ver com o Lucas

### Resultados Transformações

In [15]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,age_group,capital_net,education_age_ratio,free_time,is_married
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,Adult,2174,0.333333,128,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,Middle-Aged,0,0.260000,155,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,Adult,0,0.236842,128,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,Middle-Aged,0,0.132075,128,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,Adult,0,0.464286,128,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47616,33,Private,245211,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K.,Adult,0,0.393939,128,0
47617,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.,Adult,0,0.333333,132,0
47618,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.,Adult,0,0.342105,118,1
47619,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.,Adult,5455,0.295455,128,0


## Machine Learning

In [ ]:
# TODO: Usar aqui o que vai estar no `train.py`, e posteriormente criar o `test.py`.